In [24]:
# Import library
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import *

In [7]:
# Load dataset
df = spark.read.load("/content/vgsales.csv",format="csv", sep=",", inferSchema="true", header="true")
df.show(5)

+----+--------------------+--------+----+------------+---------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+---------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports| Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform| Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing| Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports| Nintendo|   15.75|   11.01|    3.28|       2.96|        33.0|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing| Nintendo|   11.27|    8.89|   10.22|        1.0|       31.37|
+----+--------------------+--------+----+------------+---------+

In [8]:
# Check dataframe schema
df.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



In [15]:
# Check missing values
df_null = {col:df.filter(df[col].isNull()).count() for col in df.columns}
df_null

{'Rank': 0,
 'Name': 0,
 'Platform': 0,
 'Year': 0,
 'Genre': 0,
 'Publisher': 0,
 'NA_Sales': 0,
 'EU_Sales': 0,
 'JP_Sales': 0,
 'Other_Sales': 0,
 'Global_Sales': 0}

In [11]:
# Describe the dataset
df.describe().show()

+-------+-----------------+--------------------+--------+------------------+--------+---------------+-------------------+------------------+-------------------+--------------------+------------------+
|summary|             Rank|                Name|Platform|              Year|   Genre|      Publisher|           NA_Sales|          EU_Sales|           JP_Sales|         Other_Sales|      Global_Sales|
+-------+-----------------+--------------------+--------+------------------+--------+---------------+-------------------+------------------+-------------------+--------------------+------------------+
|  count|            16598|               16598|   16598|             16598|   16598|          16598|              16598|             16598|              16598|               16598|             16598|
|   mean|8300.605253645017|              1942.0|  2600.0|2006.4064433147546|    NULL|           NULL|0.26466742981084057|0.1466520062658483|0.07778166044101108|0.048063019640913515|  0.53744065550

In [16]:
# Create database table
df.createOrReplaceTempView("vgsales")

In [42]:
# Total Sales by Order Year
q = """
  SELECT
    Year,
    ROUND(SUM(NA_Sales),2) AS `Total NA Sales`,
    ROUND(SUM(EU_Sales),2) AS `Total EU Sales`,
    ROUND(SUM(JP_Sales),2) AS `Total JP Sales`,
    ROUND(SUM(Other_Sales),2) AS `Total Other Sales`,
    ROUND(SUM(Global_Sales),2) AS `Total Global Sales`
  FROM vgsales
  GROUP BY Year
  ORDER BY Year;
"""
spark.sql(q).show()

+----+--------------+--------------+--------------+-----------------+------------------+
|Year|Total NA Sales|Total EU Sales|Total JP Sales|Total Other Sales|Total Global Sales|
+----+--------------+--------------+--------------+-----------------+------------------+
|1980|         10.59|          0.67|           0.0|             0.12|             11.38|
|1981|          33.4|          1.96|           0.0|             0.32|             35.77|
|1982|         26.92|          1.65|           0.0|             0.31|             28.86|
|1983|          7.76|           0.8|           8.1|             0.14|             16.79|
|1984|         33.28|           2.1|         14.27|              0.7|             50.36|
|1985|         33.73|          4.74|         14.56|             0.92|             53.94|
|1986|          12.5|          2.84|         19.81|             1.93|             37.07|
|1987|          8.46|          1.41|         11.63|              0.2|             21.74|
|1988|         23.87|

In [44]:
# Total Sales by Platform
q = """
  SELECT
    Platform,
    ROUND(SUM(NA_Sales),2) AS `Total NA Sales`,
    ROUND(SUM(EU_Sales),2) AS `Total EU Sales`,
    ROUND(SUM(JP_Sales),2) AS `Total JP Sales`,
    ROUND(SUM(Other_Sales),2) AS `Total Other Sales`,
    ROUND(SUM(Global_Sales),2) AS `Total Global Sales`
  FROM vgsales
  GROUP BY Platform
  ORDER BY Platform;
"""
spark.sql(q).show()

+--------+--------------+--------------+--------------+-----------------+------------------+
|Platform|Total NA Sales|Total EU Sales|Total JP Sales|Total Other Sales|Total Global Sales|
+--------+--------------+--------------+--------------+-----------------+------------------+
|    2600|          90.6|          5.47|           0.0|             0.91|             97.08|
|     3DO|           0.0|           0.0|           0.1|              0.0|               0.1|
|     3DS|         78.87|         58.52|         97.35|            12.63|            247.46|
|      DC|          5.43|          1.69|          8.56|             0.27|             15.97|
|      DS|        390.71|        194.65|        175.57|            60.53|            822.49|
|      GB|        114.32|         47.82|         85.12|              8.2|            255.45|
|     GBA|        187.54|         75.25|         47.33|             7.73|             318.5|
|      GC|        133.46|         38.71|         21.58|             5.

In [50]:
# Total Sales by Publisher
q = """
  SELECT
    Publisher,
    ROUND(SUM(NA_Sales),2) AS `Total NA Sales`,
    ROUND(SUM(EU_Sales),2) AS `Total EU Sales`,
    ROUND(SUM(JP_Sales),2) AS `Total JP Sales`,
    ROUND(SUM(Other_Sales),2) AS `Total Other Sales`,
    ROUND(SUM(Global_Sales),2) AS `Total Global Sales`
  FROM vgsales
  GROUP BY Publisher
  ORDER BY Publisher;
"""
spark.sql(q).show()

+--------------------+--------------+--------------+--------------+-----------------+------------------+
|           Publisher|Total NA Sales|Total EU Sales|Total JP Sales|Total Other Sales|Total Global Sales|
+--------------------+--------------+--------------+--------------+-----------------+------------------+
|     10TACLE Studios|          0.07|          0.04|           0.0|              0.0|              0.11|
|          1C Company|          0.01|          0.07|           0.0|             0.02|               0.1|
|20th Century Fox ...|          1.82|           0.1|           0.0|             0.01|              1.94|
|              2D Boy|           0.0|          0.03|           0.0|             0.01|              0.04|
|                 3DO|          6.48|          3.04|           0.0|             0.63|             10.12|
|             49Games|           0.0|          0.04|           0.0|              0.0|              0.04|
|           505 Games|         31.83|         16.43|   

In [54]:
# Total Sales by Genre
q = """
  SELECT
    Genre,
    ROUND(SUM(NA_Sales),2) AS `Total NA Sales`,
    ROUND(SUM(EU_Sales),2) AS `Total EU Sales`,
    ROUND(SUM(JP_Sales),2) AS `Total JP Sales`,
    ROUND(SUM(Other_Sales),2) AS `Total Other Sales`,
    ROUND(SUM(Global_Sales),2) AS `Total Global Sales`
  FROM vgsales
  GROUP BY Genre
  ORDER BY Genre;
"""
spark.sql(q).show()

+------------+--------------+--------------+--------------+-----------------+------------------+
|       Genre|Total NA Sales|Total EU Sales|Total JP Sales|Total Other Sales|Total Global Sales|
+------------+--------------+--------------+--------------+-----------------+------------------+
|      Action|        877.83|         525.0|        159.95|           187.38|           1751.18|
|   Adventure|         105.8|         64.13|         52.07|            16.81|            239.04|
|    Fighting|        223.59|        101.32|         87.35|            36.68|            448.91|
|        Misc|        410.24|        215.98|        107.76|            75.32|            809.96|
|    Platform|        447.05|        201.63|        130.77|            51.59|            831.37|
|      Puzzle|        123.78|         50.78|         57.31|            12.55|            244.95|
|      Racing|        359.42|        238.39|         56.69|            77.27|            732.04|
|Role-Playing|        327.28| 

In [55]:
# Total Global Sales by Platform & Genre
q = """
  SELECT
    Platform,
    Genre,
    ROUND(SUM(Global_Sales),2) AS `Total Global Sales`
  FROM vgsales
  GROUP BY Platform, Genre
  ORDER BY `Total Global Sales` DESC;
"""
spark.sql(q).show()

+--------+------------+------------------+
|Platform|       Genre|Total Global Sales|
+--------+------------+------------------+
|     PS3|      Action|            307.88|
|     Wii|      Sports|            292.06|
|    X360|     Shooter|            278.55|
|     PS2|      Sports|            273.41|
|     PS2|      Action|            272.76|
|    X360|      Action|            242.67|
|     Wii|        Misc|            221.06|
|     PS3|     Shooter|            196.04|
|     PS2|      Racing|            156.28|
|    X360|      Sports|            139.12|
|      DS|        Misc|            137.76|
|     PS3|      Sports|            135.56|
|      DS|  Simulation|            132.03|
|      PS|      Action|            127.05|
|      DS|Role-Playing|            126.85|
|      PS|      Sports|            120.97|
|     Wii|      Action|            118.58|
|      DS|      Action|            115.56|
|     PS2|     Shooter|            108.57|
|      PS|      Racing|            103.19|
+--------+-

In [56]:
# Total Global Sales by Publisher & Genre
q = """
  SELECT
    Publisher,
    Genre,
    ROUND(SUM(Global_Sales),2) AS `Total Global Sales`
  FROM vgsales
  GROUP BY Publisher, Genre
  ORDER BY `Total Global Sales` DESC;
"""
spark.sql(q).show()

+--------------------+------------+------------------+
|           Publisher|       Genre|Total Global Sales|
+--------------------+------------+------------------+
|     Electronic Arts|      Sports|            479.67|
|            Nintendo|    Platform|            427.21|
|          Activision|     Shooter|            299.87|
|            Nintendo|Role-Playing|             284.9|
|            Nintendo|      Sports|            218.01|
|Take-Two Interactive|      Action|            211.08|
|            Nintendo|        Misc|            180.67|
|     Electronic Arts|     Shooter|            158.26|
|            Nintendo|      Racing|             151.3|
|     Electronic Arts|      Racing|            145.77|
|             Ubisoft|      Action|            142.94|
|          Activision|      Action|            142.33|
|            Nintendo|      Action|            128.18|
|            Nintendo|      Puzzle|            124.88|
|Warner Bros. Inte...|      Action|            118.24|
|     Elec

In [58]:
# Total Global Sales by Platform & Genre order on 1999 and publisher is nintendo
q = """
  SELECT
    Platform,
    Genre,
    ROUND(SUM(Global_Sales),2) AS `Total Global Sales`
  FROM vgsales
  WHERE Year = 1999 AND Publisher = 'Nintendo'
  GROUP BY Platform, Genre
  ORDER BY `Total Global Sales` DESC;
"""
spark.sql(q).show()

+--------+------------+------------------+
|Platform|       Genre|Total Global Sales|
+--------+------------+------------------+
|      GB|Role-Playing|              23.1|
|      GB|        Misc|              6.53|
|     N64|    Strategy|              5.72|
|     N64|    Fighting|              5.55|
|     N64|    Platform|              5.27|
|      GB|    Platform|              5.07|
|     N64|  Simulation|              3.63|
|     N64|      Racing|              3.52|
|     N64|        Misc|               2.5|
|     N64|      Action|              1.47|
|     N64|     Shooter|              1.16|
|     N64|      Sports|              0.54|
|     N64|Role-Playing|              0.41|
|      GB|      Action|              0.31|
|     N64|      Puzzle|              0.29|
|    SNES|    Strategy|              0.26|
+--------+------------+------------------+



In [59]:
# Total Global Sales by Publisher & Genre order between 1980 and 1985 along with genre is sports
q = """
  SELECT
    Publisher,
    Genre,
    ROUND(SUM(Global_Sales),2) AS `Total Global Sales`
  FROM vgsales
  WHERE Year BETWEEN 1980 AND 1985 AND Genre = 'Sports'
  GROUP BY Publisher, Genre
  ORDER BY `Total Global Sales` DESC;
"""
spark.sql(q).show()

+------------------+------+------------------+
|         Publisher| Genre|Total Global Sales|
+------------------+------+------------------+
|          Nintendo|Sports|             11.34|
|             Atari|Sports|               1.1|
|        Activision|Sports|              1.04|
|Mattel Interactive|Sports|              0.19|
+------------------+------+------------------+

